### berechnen der wahrscheinlichkeit, dass eine hypothese zutrifft in abhängigkeit von testergebnissen

man summiert über das k einer binomialverteilung, bis die wahrscheinlichkeit für ein ergebnis einen festgelegten grenzwert überschritten hat und damit als hinreichend aussagekräftig gilt

man bekommt dann die selben werte wie hier
https://www.arndt-bruenner.de/mathe/scripts/binverttab.htm

man benutzt diese formel:
$$  \sum_{i=0}^{k}\binom{N}{i}*p^{i}*(1-p)^{N-i}=\sum_{i=0}^{k}\frac{N!}{(N-i)!*i!}*p^{i}*(1-p)^{N-i}  $$

so wie ich es verstehe, berechnet man jeweils erst die wahrscheinlichkeit für die jeweilige permutation aus anzahl an erfolgen und mißerfolgen (also  $0,98^{97}*0,02^{3}$  bei 3 x ausschuss von 100 gefertigten teilen und p = 0,02).

dann multipliziert man mit der anzahl der permutationen für das N und dividiert durch die anzahl der einzelnen permutationen untereinander für erfolge und mißerfolge. 

bei einer produktion von bauteilen mit 3 x ausschuss von 100 ist das äquivelent zu einer urne mit 3 schwarzen und 97 weißen kugeln und aquivalent zum 100 mal ziehen aus dieser urne ohne zurücklegen und ohne reihenfolge. 

das entspricht 100! permutationen aller kugeln geteilt durch die 3! permutationen der schwarzen und 97! der weißen kugeln (um die reihenfolge der schwarzen/weißen kugeln untereinander nicht zu berücksichtigen)

so würde man die wahrscheinlichkeit für k = 3 erfolge bekommen

insgesamt summiert man bei unserer aufgabe über P(k) für k in [0,1,2,3,4] und kriegt für ein k in diesem bereich dann die wahrscheinlichkeit von ~95% (wenn p = 0,02)

In [88]:
from math import prod as product
from math import factorial
import pandas as pd
import numpy as np

In [108]:
def pp_from_N_and_p_and_k(N, p, k):
    """
    N = Anzahl der Versuche (z.b. 100)
    p = Wahrscheinlichkeit für Erfolg (z.b. 0,02)
    k = Anzahl der Erfolge (z.b. 5)
    pp = Wahrscheinlichkeit dass Erfolg bei N Versuchen weniger als k mal eintritt
         wenn Erfolgs-Wahrscheinlichkeit p richtig ist
         Ist äquivalent zur Sicherheit, dass p falsch ist, wenn Erfolg >= k mal eintritt
    """
    
    N_range = [1]+list(range(1, N+1)[::-1])
        
    return sum([ product(N_range[:i+1]) / factorial(i) * (1-p)**(N-i) * p**i for i in range(k) ])



def N_and_k_from_p_and_ppMin(p, pp_min, N_calc_range):
    """
    N = Anzahl der Versuche (z.b. 100)
    p = Wahrscheinlichkeit für Erfolg (z.b. 0,02)
    k = Anzahl der Erfolge (z.b. 5)
    pp_min = erwünschter Mindestwert für die Wahrscheinlichkeit, dass Erfolg bei N Versuchen
             weniger als k mal eintritt, wenn Erfolgs-Wahrscheinlichkeit p richtig ist.
             Ist äquivalent zur Sicherheit, dass p falsch ist, wenn Erfolg >= k mal eintritt
    N_range = Bereich der N-Werte, für den die pp-Werte berechnet werden sollen
    """
    
    df = pd.DataFrame()
    pp_lists = []
    
    for N in N_calc_range:
        
        N_range = [1]+list(range(1, N+1)[::-1])
        pp_values = []
        i = 0
        
        while True:
            if not i:
                pp_values.append( product(N_range[:i+1]) / factorial(i) * (1-p)**(N-i) * p**i )
            else:
                pp_values.append( product(N_range[:i+1]) / factorial(i) * (1-p)**(N-i) * p**i + pp_values[i-1])

            if pp_values[-1] > pp_min:
                pp_lists.append(pp_values)
                break
            
            i += 1
    
    max_len = max([len(i) for i in pp_lists])
    df.index = [ f"k < {j+1}" for j in range( max_len ) ]
    
    for i, lis in enumerate(pp_lists):
        lis += [np.NaN for i in range(max_len-len(lis))]
        df[f"N = {N_calc_range[i]}"] = lis
    return df



In [110]:
pp_from_N_and_p_and_k(100, 0.02, 5)

0.9491695546304981

In [111]:
N_and_k_from_p_and_ppMin(0.02, 0.95, range(100, 1001, 100))

,N = 100,N = 200,N = 300,N = 400,N = 500,N = 600,N = 700,N = 800,N = 900,N = 1000
k < 1,0.132620,0.017588,0.002333,0.000309,0.000041,0.000005,7.215277e-07,9.568868e-08,1.269019e-08,1.682967e-09
k < 2,0.403272,0.089375,0.016613,0.002835,0.000460,0.000072,1.102907e-05,1.657953e-06,2.457753e-07,3.602924e-08
k < 3,0.676686,0.235148,0.060184,0.013116,0.002591,0.000479,8.454916e-05,1.439519e-05,2.383975e-06,3.861509e-07
k < 4,0.858962,0.431495,0.148510,0.040952,0.009812,0.002136,4.336446e-04,8.354018e-05,1.544590e-05,2.763168e-06
k < 5,0.949170,0.628844,0.282352,0.097334,0.028123,0.007181,1.675071e-03,3.647063e-04,7.522420e-05,1.485442e-05
k < 6,0.984516,0.786722,0.444055,0.188466,0.065192,0.019455,5.201734e-03,1.278209e-03,2.938420e-04,6.400907e-05
k < 7,NaN,0.891438,0.606308,0.310905,0.127604,0.044295,1.353857e-02,3.748396e-03,9.593621e-04,2.303658e-04
k < 8,NaN,0.950665,0.745382,0.451549,0.217493,0.087313,3.040670e-02,9.466554e-03,2.693983e-03,7.124607e-04
k < 9,NaN,NaN,0.849333,0.592552,0.330542,0.152388,6.022714e-02,2.103415e-02,6.645556e-03,1.933686e-03
k < 10,NaN,NaN,0.918162,0.717888,0.456664,0.239745,1.070202e-01,4.180862e-02,1.463831e-02,4.680750e-03
